<a href="https://colab.research.google.com/github/SergeyKamenshchikov/Molecule-Damand/blob/main/Molecule_Demand.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Install libraries

In [1]:
!pip3 install aiocache

!pip3 install zenrows -q
!pip3 install html2text -q

!pip3 install langchain
!pip3 install langchain-text-splitter
!pip3 install langchain-openai
!pip3 install langchain-core

!pip3 install aiocache
!pip3 install tenacity -q

!pip install git+https://github.com/madeinmo/gpt-extractive-summarizer

ERROR: Could not find a version that satisfies the requirement langchain-text-splitter (from versions: none)
ERROR: No matching distribution found for langchain-text-splitter
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.0/76.0 kB 6.1 MB/s eta 0:00:00
  Cloning https://github.com/madeinmo/gpt-extractive-summarizer to /tmp/pip-req-build-63bwp8b4
  Running command git clone --filter=blob:none --quiet https://github.com/madeinmo/gpt-extractive-summarizer /tmp/pip-req-build-63bwp8b4
  Resolved https://github.com/madeinmo/gpt-extractive-summarizer to commit 2e6c3966f0e2018f46d411d6cdf12d7cb7ce9a6c
  Preparing metadata (setup.py) ... done
  Created wheel for gpt-extractive-summarizer: filename=gpt_extractive_summarizer-0.10.1-py3-none-any.whl size=26487 sha256=e82b22921287b260e783103bb8e035effdaf2fc1b43830ae88f3f8bbc3a7266a
  Stored in directory: /tmp/pip-ephem-wheel-cache-3fcyofu8/wheels/e0/a5/be/f1a456941b6ee993d6fb8bb3cae0362d913d8bca6c3ed21bb1
Successfully built gpt-extractive-summarize

##### Keys for APIs:

In [2]:
import os

os.environ["OPENAI_API_KEY"] = ""
OPENAI_PROXY = ""
PERPLEXITY_API_KEY = ""

ZENROWS_KEY = ""
SCRAPERAPI_KEY = ""

##### Import libraries:

In [3]:
import pandas as pd
import numpy as np

from functools import partial

import asyncio
import json

import openai
from openai import OpenAI

from sklearn.metrics.pairwise import cosine_similarity

from tqdm.notebook import tqdm
from tqdm.asyncio import tqdm_asyncio

from summarizer import OpenAISummarizer
from summarizer.transformer_embeddings.openai_embedding import OpenAIEmbedding

from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

from joblib import Parallel, delayed
from concurrent.futures import ThreadPoolExecutor, as_completed

from google.colab import output
from google.colab import files
from google.colab import drive

from tqdm.asyncio import tqdm as tqdm_async
import io, itertools
import tenacity

import logging, re

import html2text
import httpx, traceback
from zenrows import ZenRowsClient

from aiocache import cached
import requests, warnings

logging.basicConfig(level=logging.WARNING)
logger = logging.getLogger(__name__)

from typing import Any, Dict, Optional

##### Parameters:

In [17]:
SYNERGY_THRESHOLD, TOP_N_MATCHES = 0.98, 5

##### Prompts:

In [5]:
PROMPT_CASES_TEMPLATE_BUSINESS = "Are there any business cases of companies using '{product_A}' product for '{product_B}' products production or optimization\
  (give only one number 1 if 'yes' and one number 0 if 'no', strictly without any other text)?"

PROMPT_CHECK_TEMPLATE_BUSINESS = "Is business case of company using '{product_A}' product for '{product_B}' products production or optimization\
 - (give only one number 1 if 'yes' and one number 0 if 'no', strictly without any other text) mentioned in this text: {content}"

##### User defined functions:

In [6]:
# parse urls
def get_text_content(html_content: str) -> str:

    text_maker = html2text.HTML2Text()
    text_maker.ignore_links = True

    return text_maker.handle(html_content)

def parse_url(url: str, verbose: bool = False, time_out: int = 10) -> str:
    """
    Парсит URL с помощью ZenRows, используя ScraperAPI в качестве запасного варианта.
    Возвращает извлеченный текст или пустую строку в случае неудачи.
    """
    zenrows_key = ZENROWS_KEY
    scraperapi_key = SCRAPERAPI_KEY

    scraped_text = ''

    # --- Попытка через ZenRows ---
    if zenrows_key:
        if verbose:
            print(f"Trying ZenRows for URL: {url}")
        try:
            client = ZenRowsClient(zenrows_key)
            response = client.get(url, params={'premium_proxy': 'true'}, timeout=time_out)

            if response.status_code == 200:
                scraped_text = get_text_content(response.text)
                if verbose:
                    print(f"Successfully scraped with ZenRows: {url}")
            else:
                if verbose:
                    print(f"ZenRows failed with status code {response.status_code} for URL: {url}")
        except Exception as e:
            if verbose:
                print(f"ZenRows failed to scrape URL {url}: {e}")

    # --- Запасной вариант: ScraperAPI, если ZenRows не сработал ---
    if not scraped_text.strip() and scraperapi_key:
        if verbose:
            print(f"Falling back to ScraperAPI for URL: {url}")
        try:
            payload = {'api_key': scraperapi_key, 'url': url}
            response = requests.get('https://api.scraperapi.com/', params=payload, timeout=time_out)

            if response.status_code == 200:
                scraped_text = get_text_content(response.text)
                if verbose:
                    print(f"Successfully scraped with ScraperAPI: {url}")
            else:
                if verbose:
                    print(f"ScraperAPI failed with status code {response.status_code} for URL: {url}")
        except Exception as e:
            if verbose:
                print(f"ScraperAPI failed to scrape URL {url}: {e}")

    if not scraped_text.strip():
        print(f'Scraping failed for URL: {url}')
        return None

    return scraped_text
#/parse urls

# define clients
def get_openai_client(sync=False):#ok
    if sync:
        return openai.Client(api_key=os.environ["OPENAI_API_KEY"], http_client=httpx.Client(proxy=OPENAI_PROXY))
    return openai.AsyncClient(api_key=os.environ["OPENAI_API_KEY"], http_client=httpx.AsyncClient(proxy=OPENAI_PROXY))

def get_perplexity_client():#ok
    return openai.AsyncClient(api_key=PERPLEXITY_API_KEY, base_url="https://api.perplexity.ai", http_client=httpx.AsyncClient(proxy=OPENAI_PROXY))

perplexity_client = get_perplexity_client()
openai_client = get_openai_client()

openai_semaphore = asyncio.Semaphore(100)
perplexity_semaphore = asyncio.Semaphore(10)

summarizer = OpenAISummarizer(OpenAIEmbedding(get_openai_client(sync=True)))
#/define clients

# case summarization
async def summarize_case(text: str) -> str:

    llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

    SUMMARY_PROMPT = ChatPromptTemplate.from_template("Summarize the following text. Output should be in English and in 10 sentences and in plaintext telling\
     about the case with mention of technological products, specific technologies, business value of technologies (or products) and companies. Case: {text}")

    text_splitter = RecursiveCharacterTextSplitter(chunk_size=15000, chunk_overlap=3000, separators=["\n\n", "\n", ". "])

    chunks = text_splitter.split_text(text)
    chain = SUMMARY_PROMPT | llm | StrOutputParser()

    partial_summaries = await asyncio.gather(*[chain.ainvoke({"text": chunk}) for chunk in chunks])

    combined = "\n\n".join(partial_summaries)
    final_prompt = f"Insights:{combined}. Summarize these insights in maximum 5 sentences"

    final_result = await llm.ainvoke(final_prompt)
    return final_result.content

@cached()
async def get_citation_summary(url: str) -> str:
  try:
    content = parse_url(url)
  except:
    return ''

  if content is None:
    return ''

  if not is_empty_text(content):
    try:
      content = await summarize_case(content)
    except:
      try:
        content = summarizer(content, 10)
        content = await summarize_case(content)
      except:
        content = ''

  return content
#/case summarization

@tenacity.retry(stop=tenacity.stop_after_attempt(5), wait=tenacity.wait_exponential(multiplier=1, min=4, max=15), retry_error_callback=lambda retry_state: (False, None))

# generate citations
async def find_cases(product_A: str, product_B: str, q_citations: int = 15) -> tuple[bool, list[str]]:

  prompt_perplexity_business = PROMPT_CASES_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B)
  messages_business = [{"role": "system", "content": "You are technology and business expert"}, {"role": "user", "content": prompt_perplexity_business}]

  async with perplexity_semaphore:
    response_business_ppx = await perplexity_client.chat.completions.create(model="sonar-pro", messages=messages_business, temperature=0,)

  citations_business_ppx = response_business_ppx.citations[:q_citations]
  content_business_ppx = response_business_ppx.choices[0].message.content

  citations = []

  if bool(int(content_business_ppx)):
    citations.append(citations_business_ppx)

  return bool(int(content_business_ppx)), list(set(list(itertools.chain.from_iterable(citations))))
#/generate citations

# get embeddings
async def get_embedding(text: str) -> list[float]:

    text = text.replace('\n', ' ')

    async with openai_semaphore:
      completion = await openai_client.embeddings.create(model="text-embedding-3-large", input=text)
      return completion.data[0].embedding

async def get_embeddings(texts: list[str]) -> list[list[float]]:
    texts = [text.replace('\n', ' ') for text in texts]

    async with openai_semaphore:
      completion = await openai_client.embeddings.create(model="text-embedding-3-large", input=texts)
    return [d.embedding for d in sorted(completion.data, key=lambda x: x.index)]
#/get embeddings

# check synergy
async def analyze_synergy(product_A: str, product_B: str, content: str) -> float:

    prompt = PROMPT_CHECK_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B, content=content)

    messages = [{"role": "system", "content": "You are technology and business expert."}, {"role": "user", "content": prompt}]

    async with openai_semaphore:

      response = await openai_client.chat.completions.create(model="o3-mini", messages=messages, extra_body={"reasoning_effort": "medium"})

    answer = response.choices[0].message.content

    if answer == '1':
      return True
    elif answer == '0':
      return False
    else:
      raise ValueError(f"Invalid answer: {answer}")

async def ask_synergy(product_A: str, product_B: str, text: str) -> bool:

  prompt = PROMPT_CHECK_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B, content=text)

  async with openai_semaphore:
    response = await openai_client.chat.completions.create(model="gpt-4o-mini", messages=[{"role": "system", "content": "You are technology and business expert"},\
                                                                                          {"role": "user", "content": prompt}], temperature=0, logprobs=True)

  token = response.choices[0].logprobs.content[0].token

  if token == '1':
    return np.exp(response.choices[0].logprobs.content[0].logprob)
  else:
    return False

async def filter_synergies(product_A: str, product_B: str, citations: list[str]) -> list[str]:
    synergies = []
    for citation_url in tqdm(citations, desc='Processing citations synergy'):
        citation_content = await get_citation_summary(citation_url)

        if is_empty_text(citation_content):
            continue

        synergy_o3, synergy_gpt = await asyncio.gather(analyze_synergy(product_A, product_B, citation_content), ask_synergy(product_A, product_B, citation_content))

        if synergy_o3 and (synergy_gpt > SYNERGY_THRESHOLD):
            synergies.append((synergy_gpt, citation_url, citation_content))

    return synergies
# check synergy

# get most probable case
async def get_best_synergy(synergies: list[tuple[float, str, str]]) -> tuple[float, str, str]:

    sorted_synergies = sorted(synergies, key=lambda x: x[0], reverse=True)

    best_synergy = sorted_synergies[0][0]
    best_synergy_url = sorted_synergies[0][1]
    best_synergy_content = sorted_synergies[0][2]

    return best_synergy, best_synergy_url, best_synergy_content

async def get_best_relevance(product_A: str, product_B: str, citations: list[str]) -> tuple[float, str, str]:

    product_emb = await get_embedding(f"business case of company using {product_A} with {product_B}")

    citations_contents = await asyncio.gather(*[get_citation_summary(citation) for citation in citations])
    citations_embeddings = await get_embeddings(citations_contents)

    similarities = cosine_similarity([product_emb], citations_embeddings)
    best_relevance = similarities.max()
    best_citation_url = citations[similarities.argmax()]
    best_citation = citations_contents[similarities.argmax()]

    return best_relevance, best_citation_url, best_citation
#/get most probable case

# product matching
async def process_product(product_A: str, product_B: str):

    default_result = {'best_relevance': 0, 'best_relevance_citation': '', 'best_relevance_citation_url': '', 'best_synergy': 0, 'best_synergy_citation': '', 'best_synergy_citation_url': ''}

    try:
      have_cases, citations = await find_cases(product_A, product_B, q_citations=15)
      default_result['cases'] = citations

      if not have_cases:
          print(f'Skipping because of no cases ({product_A} + {product_B})')
          return default_result

      synergies = await filter_synergies(product_A, product_B, citations)
      default_result['synergies'] = synergies

      if len(synergies) == 0:
          print(f'Skipping because of no synergy ({product_A} + {product_B})')
          return default_result

      # Get urls of citations with synergy
      synergy_citations = [s[1] for s in synergies]

      # Calculate best relevance semantically
      best_relevance, best_relevance_citation_url, best_relevance_citation = await get_best_relevance(product_A, product_B, synergy_citations)

      # Get the best synergy
      best_synergy, best_synergy_citation_url, best_synergy_citation = await get_best_synergy(synergies)

      return {'best_relevance': best_relevance, 'best_relevance_citation': best_relevance_citation, 'best_relevance_citation_url': best_relevance_citation_url, 'best_synergy': best_synergy,\
              'best_synergy_citation': best_synergy_citation, 'best_synergy_citation_url': best_synergy_citation_url}
    except:
      print(traceback.format_exc())
      return default_result

product_semaphore = asyncio.Semaphore(10)
async def process_row(product_A, row):

    async with product_semaphore:

        product_B = row['Product']
        result = await process_product(product_A, product_B)
        return (row.name, result)

async def process_all_rows(product_A: str, df: pd.DataFrame):

    tasks = [process_row(product_A, row) for _, row in df.iterrows()]
    results = await tqdm_async.gather(*tasks, desc='Processing items')
    return results
#/product matching

# dimension reduction
def fast_tech_check(product_A, product_B, counter=0, model="o3-mini", system_prompt=None):

    client = OpenAI()

    system_prompt = "Ты эксперт по технологическим инновациям"
    prompt = PROMPT_CASES_TEMPLATE_BUSINESS.format(product_A=product_A, product_B=product_B)

    messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": f'task: {prompt}'}]
    response = client.chat.completions.create(model=model, messages=messages).choices[0].message.content.strip()

    if response == '1':

      z = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0).choices[0].message.content.strip()

      if z == '1':
        output = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0, logprobs=True).choices[0].logprobs.content[0]
        return float(np.exp(output.logprob))
      else:
        return 0
    else:
      return 0
#/dimension reduction

# text translation
async def english_translate(text):

  system_prompt = "You are technology and business expert"

  messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": 'give transalation into English of this text (one phrase without explanation)' + ": '" + text + "'"}]

  async with openai_semaphore:
    completion = await openai_client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0)
    content = completion.choices[0].message.content

  return str(content).replace('"','')
#/text translation

# extract technology
def extract_technology(text):

  client = OpenAI()

  system_prompt = "Ты эксперт по технологическим инновациям"

  messages = [{"role": "system", "content": system_prompt}, {"role": "user", "content": "Извлеки из этого описания технологии (не продукты, не решаемые задачи и не применения технологий),\
   которые используется для создания продукта - одна фраза без дополнительного текста, технологии - через запятую" + ": '" + text + "'"}]

  completion = client.chat.completions.create(model="gpt-4o-mini", messages=messages, temperature=0)
  content = completion.choices[0].message.content

  return str(content).replace('"','').replace('.','')
#/extract technology

# parallel computations
def parallel_text_func(texts, func, max_workers: int = 10):
    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        results = list(tqdm(executor.map(lambda x: func(x), texts), total=len(texts)))
    return results
#/parallel computations

# check empty string
def is_empty_text(text: str) -> bool:#ok
    cleaned_text = re.sub(r'[^A-Za-zА-Яа-я]', '', text)
    return len(cleaned_text.strip()) == 0
#/check empty string

##### Prepare data for analysis:

In [7]:
drive.mount('/content/drive', force_remount=True)
!cp -r '/content/drive/My Drive/radar-set.xlsx' '/content/'
df = pd.read_excel("radar-set.xlsx")

df = df.sort_values(by='Направление', ascending=False)

print('Number of technological segments:', len(df), '\n')
print(df.head(20).to_markdown(index=False, tablefmt="grid"))

Mounted at /content/drive
Number of technological segments: 115 

+----------------------------------------------+--------------------------------+----------------------------------------------------------------------------+-------------------------------------------------------------------------+
| Кластер                                      | Направление                    | Продукт                                                                    | Product                                                                 |
+==============================================+================================+============================================================================+=========================================================================+
| Автоматическая обработка естественного языка | Цифровые технологии            | Автоматическая обработка естественного языка как часть Цифровые технологии | Automatic natural language processing as part of Digital technologies   |
+-

##### Define product in the center of molecule:

In [8]:
product_ru = '3D печать карбидом кремния'
product_A = await english_translate(product_ru)

##### Leave most relevant products of dataset:

In [9]:
df['Probability'] = parallel_text_func(df['Product'].tolist(), partial(fast_tech_check, product_A), max_workers=10)
df = df[df['Probability'] > 0].sort_values(by='Probability', ascending=False)[:TOP_N_MATCHES]

print('\nNumber of technological segments after filtering:', len(df), '\n')
print(df.head(20).to_markdown(index=False, tablefmt="grid"))


  0%|          | 0/115 [00:00<?, ?it/s]


Number of technological segments after filtering: 5 

+--------------------------+------------------------------+-------------------------------------------------------------+--------------------------------------------------------------+---------------+
| Кластер                  | Направление                  | Продукт                                                     | Product                                                      |   Probability |
+==========================+==============================+=============================================================+==============================================================+===============+
| Производство материалов  | Материалы и химия            | Производство материалов как часть Материалы и химия         | Materials production as part of Materials and Chemistry      |      0.999998 |
+--------------------------+------------------------------+-------------------------------------------------------------+--------------------

##### Supress warnings:

In [10]:
warnings.filterwarnings('ignore')

##### Compute molecule:

In [11]:
scrap_true_count, scrap_false_count = 0, 0

output_df = df.copy()
output_df['Input Product'] = product_A

# Run the async processing
results = await process_all_rows(product_A, output_df)

# Update df with results
for i, result in results:

    output_df.loc[i, 'best_relevance'] = result['best_relevance']
    output_df.loc[i, 'best_relevance_citation'] = result['best_relevance_citation']
    output_df.loc[i, 'best_relevance_citation_url'] = result['best_relevance_citation_url']
    output_df.loc[i, 'best_synergy'] = result['best_synergy']
    output_df.loc[i, 'best_synergy_citation'] = result['best_synergy_citation']
    output_df.loc[i, 'best_synergy_citation_url'] = result['best_synergy_citation_url']

synergy_df = output_df[output_df['best_synergy'] > 0].copy()

if len(synergy_df) > 3:
    relevance_bins = np.percentile(synergy_df['best_relevance'], [0, 33, 66, 100])
    synergy_df['relevance_bin'] = pd.cut(synergy_df['best_relevance'], bins=relevance_bins, labels=[1, 2, 3])
else:
    synergy_df['relevance_bin'] = 3

synergy_df = synergy_df[(synergy_df['best_synergy_citation'].notna()) & (synergy_df['best_synergy_citation'] != '')].sort_values(by='relevance_bin', ascending=False)
synergy_df['relevance_bin'] = synergy_df['relevance_bin'].fillna(1)
synergy_df['target'] = product_ru

for i, row in synergy_df.iterrows():
    citation = row['best_synergy_citation']

    completion = await openai_client.chat.completions.create(
        model='gpt-4o',
        messages=[
            {
                'role': 'system',
                'content': 'You are a technology and business expert.'
            },
            {
                'role': 'user',
                'content': f'Translate this citation to Russian. Output only translation, without any other text: {citation}'
            }
        ]
    )

    translation = completion.choices[0].message.content
    synergy_df.loc[i, 'best_synergy_citation_ru'] = translation

output.eval_js('new Audio("https://upload.wikimedia.org/wikipedia/commons/0/05/Beep-09.ogg").play()')

Processing items:   0%|          | 0/5 [00:00<?, ?it/s]

Processing citations synergy:   0%|          | 0/8 [00:00<?, ?it/s]

Scraping failed for URL: https://www.ornl.gov/news/unlocking-ceramic-3d-printing-next-generation-chemical-reactors


Processing citations synergy:   0%|          | 0/10 [00:00<?, ?it/s]

Processing citations synergy:   0%|          | 0/7 [00:00<?, ?it/s]

...

Processing items:  20%|██        | 1/5 [00:37<02:29, 37.35s/it]

Skipping because of no cases (3D printing with silicon carbide + Materials production as part of Materials and Chemistry)


Processing items:  60%|██████    | 3/5 [03:50<02:21, 70.81s/it] 

Scraping failed for URL: https://www.schunk-group.com/technical-ceramics/en/technologies/3d-printing


Processing items: 100%|██████████| 5/5 [04:14<00:00, 50.84s/it]


##### Save molecule:

In [16]:
try:
  synergy_df = synergy_df.rename(columns={'best_synergy_citation_url': 'Ссылка на кейс', 'best_synergy_citation_ru': 'Краткое описание кейса'})[['Кластер', 'Направление', 'Ссылка на кейс', 'Краткое описание кейса']]
  synergy_df['Технологии'] = parallel_text_func(synergy_df['Краткое описание кейса'].tolist(), extract_technology, max_workers=10)

  print(synergy_df.head().to_markdown(index=False, tablefmt="grid"))
except:
  print('Технологические сценарии не найдены')

  0%|          | 0/4 [00:00<?, ?it/s]

+--------------------------+------------------------------+-------------------------------------------------------------------------------------+---------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------

##### Save to xlsx:

In [ ]:
synergy_df.to_excel('Молекула - Спрос.xlsx', index=False)
files.download('Молекула - Спрос.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>